In [378]:
!pip install streamlit --quiet
!npm install localtunnel --global --silent
!pip install openpyxl --quiet

In [379]:
import pandas as pd
import streamlit as st
import os
import numpy as np
import plotly.express as px


In [380]:
df = pd.read_csv('dados.csv')

In [403]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   safra                  69 non-null     object 
 1   regiao                 69 non-null     object 
 2   municipio              69 non-null     object 
 3   latitude               69 non-null     float64
 4   longitude              69 non-null     float64
 5   altitude               69 non-null     int64  
 6   rendimento_kg_ha       69 non-null     int64  
 7   reducao_chuvas_perc    69 non-null     int64  
 8   queda_rendimento_perc  69 non-null     float64
 9   regiao_geo             69 non-null     object 
dtypes: float64(3), int64(3), object(4)
memory usage: 5.5+ KB


In [406]:
display(df.head())

,safra,regiao,municipio,latitude,longitude,altitude,rendimento_kg_ha,reducao_chuvas_perc,queda_rendimento_perc,regiao_geo
0,2016-17,Noroeste,Nova Olimpia,-23.47,-53.08,431,3000,0,0.00,Norte_Parana
1,2017-18,Noroeste,Nova Olimpia,-23.47,-53.08,431,3000,0,0.00,Norte_Parana
2,2019-20,Noroeste,Nova Olimpia,-23.47,-53.08,431,3350,0,0.00,Norte_Parana
3,2018-19,Noroeste,Nova Olimpia,-23.47,-53.08,431,2500,-23,-19.79,Norte_Parana
4,2014-15,Noroeste,Diamante do Norte,-22.63,-52.89,368,2700,0,0.00,Norte_Parana


In [383]:
colunas_novas = {
    'Safra': 'safra',
    'Regiao': 'regiao',
    'Municipio': 'municipio',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'Altitude': 'altitude',
    'Rendimento (Kg/ha)': 'rendimento_kg_ha',
    'Reduçao-chuvas (%)': 'reducao_chuvas_perc',
    'Queda Rendimento 18-19': 'queda_rendimento_perc'
}

df.rename(columns=colunas_novas, inplace=True)

In [411]:
display(df.columns.tolist())


['safra',
 'regiao',
 'municipio',
 'latitude',
 'longitude',
 'altitude',
 'rendimento_kg_ha',
 'reducao_chuvas_perc',
 'queda_rendimento_perc',
 'regiao_geo']

In [402]:
display(df.head())

,safra,regiao,municipio,latitude,longitude,altitude,rendimento_kg_ha,reducao_chuvas_perc,queda_rendimento_perc,regiao_geo
0,2016-17,Noroeste,Nova Olimpia,-23.47,-53.08,431,3000,0,0.00,Norte_Parana
1,2017-18,Noroeste,Nova Olimpia,-23.47,-53.08,431,3000,0,0.00,Norte_Parana
2,2019-20,Noroeste,Nova Olimpia,-23.47,-53.08,431,3350,0,0.00,Norte_Parana
3,2018-19,Noroeste,Nova Olimpia,-23.47,-53.08,431,2500,-23,-19.79,Norte_Parana
4,2014-15,Noroeste,Diamante do Norte,-22.63,-52.89,368,2700,0,0.00,Norte_Parana


In [386]:
def classificar_regiao(latitude):
  if latitude>= -24:
    return 'Norte_Parana'
  elif latitude < -24 and latitude >= -25.5:
    return 'Centro_Parana'
  else:
    return 'Sul_Parana'

In [387]:
df['regiao_geo'] = df['latitude'].apply(classificar_regiao)

In [388]:
print("\n Distribuição da Nova Coluna 'regiao_geo' \n")
df_distribuicao = df['regiao_geo'].value_counts().to_frame().rename_axis('Regiao Geográfica').reset_index()
df_distribuicao.columns = ['Regiao Geográfica', 'N° de Observações']
display(df_distribuicao.style.set_properties(**{'text-align': 'center'}))


 Distribuição da Nova Coluna 'regiao_geo' 



,Regiao Geográfica,N° de Observações
0,Centro_Parana,40
1,Norte_Parana,19
2,Sul_Parana,10


In [389]:
colunas_chave = ['rendimento_kg_ha', 'reducao_chuvas_perc', 'queda_rendimento_perc', 'altitude']

In [393]:
print("\n Estatística Descritivas das Variáveis Chave' \n")
format_dict = {
    'reducao_chuvas_perc': '{:.2f}%',
    'queda_rendimento_perc': '{:.2f}%',
    'rendimento_kg_ha': '{:.2f}',
    'altitude': '{:.0f}'
}
display(df[colunas_chave].describe().style.format(format_dict).set_properties(**{'text-align': 'center'}))



 Estatística Descritivas das Variáveis Chave' 



,rendimento_kg_ha,reducao_chuvas_perc,queda_rendimento_perc,altitude
count,69.00,69.00%,69.00%,69
mean,3223.20,-3.58%,-4.10%,631
std,576.49,7.85%,11.10%,297
min,1500.00,-26.00%,-54.32%,235
25%,2975.00,0.00%,0.00%,368
50%,3339.00,0.00%,0.00%,596
75%,3600.00,0.00%,0.00%,956
max,4120.00,0.00%,0.00%,1110


In [391]:
print('\n Análise de Correlação (Fator Crítico vs. Queda de Rendimento) \n')
correlacao = df[['reducao_chuvas_perc', 'queda_rendimento_perc']].corr()
display(correlacao.style.format('{:.3f}').set_properties(**{'text-align': 'center'}))


 Análise de Correlação (Fator Crítico vs. Queda de Rendimento) 



,reducao_chuvas_perc,queda_rendimento_perc
reducao_chuvas_perc,1.000,0.891
queda_rendimento_perc,0.891,1.000


In [392]:
print('\n Mapa Interativo: Queda de Rendimento por Localização\n')

fig = px.scatter_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    color='queda_rendimento_perc',
    size='rendimento_kg_ha',
    hover_name='municipio',
    hover_data={
        'rendimento_kg_ha': ':.2f} kg/ha',
        'queda_rendimento_perc': ':.2f}%',
        'reducao_chuvas_perc': ':.2f}%',
        'latitude': False,
        'longitude': False
    },
    title='Impacto da seca (Queda de Rendimento %) em Cultivo de Soja (Paraná)',
    color_continuous_scale=px.colors.sequential.Reds,
    zoom=6.5,
    height=600
    )
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={'r':0, 't':50, 'l':0, 'b':0})
fig.show()


 Mapa Interativo: Queda de Rendimento por Localização

